<a href="https://colab.research.google.com/github/Heedou/-/blob/main/%EA%B3%A0%EB%A0%A4%EB%8C%80%EA%B3%BC%EC%A0%9C2_friends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 1.5MB 18.7MB/s 
     |████████████████████████████████| 890kB 55.0MB/s 
     |████████████████████████████████| 2.9MB 53.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=4491383f4c7cce5896cfb46403defb53485f1c3ee3ea3ca0610ac9115366337e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import tensorflow as tf
from transformers import *
import pandas as pd
import numpy as np
import json
import os

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
os.listdir('/content/gdrive/My Drive/Colab Notebooks/naver_sentiment/Friends')

['friends_train.json',
 'friends_dev.json',
 'friends_test.json',
 'best_model.h5']

In [5]:
#json 파일 형태의 friends 데이터 불러오기
with open('/content/gdrive/My Drive/Colab Notebooks/naver_sentiment/Friends/friends_train.json', 'r') as f:
  train_friends_dic = json.load(f)

with open('/content/gdrive/My Drive/Colab Notebooks/naver_sentiment/Friends/friends_test.json', 'r') as f:
  test_friends_dic = json.load(f)

In [6]:
# 데이터 프레임 형태로 고치기
data_all = []
for i in range(len(train_friends_dic)):
  for j in range(len(train_friends_dic[i])):
    data = (train_friends_dic[i][j]['utterance'], train_friends_dic[i][j]['emotion'])
    data_all.append(data)

train_friends = pd.DataFrame(data_all)
train_friends.columns = ['text', 'label']
train_friends = train_friends.dropna()

data_all2 = []
for i in range(len(test_friends_dic)):
  for j in range(len(test_friends_dic[i])):
    data = (test_friends_dic[i][j]['utterance'], test_friends_dic[i][j]['emotion'])
    data_all2.append(data)

test_friends = pd.DataFrame(data_all2)
test_friends.columns = ['text', 'label']
test_friends = test_friends.dropna()

In [7]:
# 알파벳 이외의 문자 제거하는 정제작업
train_friends_text = [i for i in train_friends['text']]
train_cleaned_friends_texts = []
for text in train_friends_text:
  try:
    text = re.sub('[^a-zA-Z\\s]', '', text)
  except:
    pass

  train_cleaned_friends_texts.append(text)

test_friends_text = [i for i in test_friends['text']]
test_cleaned_friends_texts = []
for text in test_friends_text:
  try:
    text = re.sub('[^a-zA-Z\\s]', '', text)
  except:
    pass

  test_cleaned_friends_texts.append(text)

In [8]:
len_friends = [len(i) for i in train_cleaned_friends_texts if type(i)==str]
print('friends 리뷰 길이의 최대값 : {}'.format(np.max(len_friends)))
print('friends 리뷰 길이의 최소값 : {}'.format(np.min(len_friends)))
print('friends 리뷰 길이의 평균 : {}'.format(np.mean(len_friends)))

friends 리뷰 길이의 최대값 : 327
friends 리뷰 길이의 최소값 : 1
friends 리뷰 길이의 평균 : 39.68478363791308


In [ ]:
train_friends

In [ ]:
# friends 데이터의 경우, label을 숫자로 바꾸어 주어야 함. 우선 label의 개수 확인
train_labels = [i for i in train_friends['label']]
train_labels = list(set(train_labels))
train_labels
# label 확인 결과, disgust, anger, fear, sadness, non-neutral, neutral, surprise, joy 등의 8개의 감정이 존재
# 각각의 감정에 해당하는 label을 숫자로 대체하기
for i in range(len(train_friends)):
  if train_friends['label'][i] == 'non-neutral':
    train_friends['label'][i] = 0
  elif train_friends['label'][i] == 'neutral':
    train_friends['label'][i] = 1
  elif train_friends['label'][i] == 'joy':
    train_friends['label'][i] = 2
  elif train_friends['label'][i] == 'sadness':
    train_friends['label'][i] = 3
  elif train_friends['label'][i] == 'fear':
    train_friends['label'][i] = 4
  elif train_friends['label'][i] == 'anger':
    train_friends['label'][i] = 5
  elif train_friends['label'][i] == 'surprise':
    train_friends['label'][i] = 6
  elif train_friends['label'][i] == 'disgust':
    train_friends['label'][i] = 7

In [ ]:
# friends 데이터의 경우, label을 숫자로 바꾸어 주어야 함. 우선 label의 개수 확인
test_labels = [i for i in test_friends['label']]
test_labels = list(set(test_labels))
test_labels
# label 확인 결과, disgust, anger, fear, sadness, non-neutral, neutral, surprise, joy 등의 8개의 감정이 존재
# 각각의 감정에 해당하는 label을 숫자로 대체하기
for i in range(len(test_friends)):
  if test_friends['label'][i] == 'non-neutral':
    test_friends['label'][i] = 0
  elif test_friends['label'][i] == 'neutral':
    test_friends['label'][i] = 1
  elif test_friends['label'][i] == 'joy':
    test_friends['label'][i] = 2
  elif test_friends['label'][i] == 'sadness':
    test_friends['label'][i] = 3
  elif test_friends['label'][i] == 'fear':
    test_friends['label'][i] = 4
  elif test_friends['label'][i] == 'anger':
    test_friends['label'][i] = 5
  elif test_friends['label'][i] == 'surprise':
    test_friends['label'][i] = 6
  elif test_friends['label'][i] == 'disgust':
    test_friends['label'][i] = 7

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
tokens = []
masks = []
segments = []
labels = []
max_sequence_length = 64
for i in range(len(train_friends)):
  text = str(train_friends['text'][i])
  label = train_friends['label'][i]

  #token 만드는 부분
  token = tokenizer.encode(text, max_length=max_sequence_length, pad_to_max_length=True)
  for index, value in enumerate(token):
    if value == 1:
        token[index] = 0

  #mask 만드는 부분
  zeros = token.count(0)
  mask = [1]*(max_sequence_length-zeros) + [0]*zeros

  #segment 만드는 부분
  segment = [0]*max_sequence_length


  #각 리스트에 저장
  tokens.append(token)
  masks.append(mask)
  segments.append(segment)
  labels.append(label)

# numpy array로 변경
tokens = np.array(tokens)
masks = np.array(masks)
segments = np.array(segments)
labels = np.array(labels)

train_x = [tokens, masks, segments]
train_y = labels

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
test_tokens = []
test_masks = []
test_segments = []
test_labels = []
max_sequence_length = 64
for i in range(len(test_friends)):
  text = str(test_friends['text'][i])
  label = test_friends['label'][i]
  #token 만드는 부분
  token = tokenizer.encode(text, max_length=max_sequence_length,  pad_to_max_length=True)
  for index, value in enumerate(token):
    if value == 1:
        token[index] = 0

  #mask 만드는 부분
  zeros = token.count(0)
  mask = [1]*(max_sequence_length-zeros) + [0]*zeros

  #segment 만드는 부분
  segment = [0]*max_sequence_length


  #각 리스트에 저장
  test_tokens.append(token)
  test_masks.append(mask)
  test_segments.append(segment)
  test_labels.append(label)

# numpy array로 변경
test_tokens = np.array(test_tokens)
test_masks = np.array(test_masks)
test_segments = np.array(test_segments)
test_labels = np.array(test_labels)

test_x = [test_tokens, test_masks, test_segments]
test_y = test_labels

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
model = TFBertModel.from_pretrained('bert-base-multilingual-cased', from_pt=True)
input_tokens = tf.keras.layers.Input((max_sequence_length, ), dtype=tf.int32, name='input_word_ids')
input_masks = tf.keras.layers.Input((max_sequence_length, ), dtype=tf.int32, name='input_masks')
input_segments = tf.keras.layers.Input((max_sequence_length, ), dtype=tf.int32, name='input_segment')
output = model([input_tokens, input_masks, input_segments])
output = output[0]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

In [ ]:
#embedding = tf.keras.layers.Embedding(768,128)(output)
lstm = tf.keras.layers.LSTM(128)(output)
drop = tf.keras.layers.Dropout(0.5)(lstm)
dense = tf.keras.layers.Dense(8, activation='softmax', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(drop)
final_model = tf.keras.Model([input_tokens, input_masks, input_segments], dense)
final_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00001), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics = ['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
#model_checkpoint = ModelCheckpoint('/content/gdrive/My Drive/Colab Notebooks/naver_sentiment/Friends/best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_weights_only=True)

In [ ]:
final_model.fit(train_x, train_y, epochs=30, callbacks=[early_stopping], shuffle=True, batch_size=64, validation_data=(test_x, test_y))

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Epoch 1/30


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


166/166 [==============================] - ETA: 0s - loss: 1.7639 - accuracy: 0.4507

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


166/166 [==============================] - 164s 876ms/step - loss: 1.7632 - accuracy: 0.4508 - val_loss: 1.4717 - val_accuracy: 0.5365
Epoch 2/30
166/166 [==============================] - 148s 895ms/step - loss: 1.4823 - accuracy: 0.5280 - val_loss: 1.3949 - val_accuracy: 0.5546
Epoch 3/30
166/166 [==============================] - 148s 894ms/step - loss: 1.4051 - accuracy: 0.5619 - val_loss: 1.3337 - val_accuracy: 0.5742
Epoch 4/30
166/166 [==============================] - 148s 894ms/step - loss: 1.3267 - accuracy: 0.5911 - val_loss: 1.2968 - val_accuracy: 0.5792
Epoch 5/30
166/166 [==============================] - 149s 895ms/step - loss: 1.2463 - accuracy: 0.6241 - val_loss: 1.2661 - val_accuracy: 0.5825
Epoch 6/30
166/166 [==============================] - 148s 894ms/step - loss: 1.1905 - accuracy: 0.6419 - val_loss: 1.2852 - val_accuracy: 0.5713
Epoch 7/30
166/166 [==============================] - 149s 895ms/step - loss: 1.1623 - accuracy: 0.6494 - val_loss: 1.2496 - val_accura

In [ ]:
#제출할 데이터
submit_friends = pd.read_table('/content/gdrive/My Drive/Colab Notebooks/naver_sentiment/제출데이터/en_data.csv', delimiter=',')
submit_friends.drop(['i_dialog', 'i_utterance', 'speaker'], axis='columns', inplace=True)
submit_friends

,id,utterance
0,0,"Alright, whadyou do with him?"
1,1,Oh! You're awake!
2,2,Then you gotta come clean with Ma! This is not...
3,3,"Yeah, but this is"
4,4,I don't wanna hear it! Now go to my room!
...,...,...
1618,1618,Nooo.
1619,1619,"Hi, Kate!"
1620,1620,"Hi, Lauren."
1621,1621,"Hi, Lauren."


In [ ]:
submit_tokens = []
submit_masks = []
submit_segments = []
max_sequence_length = 64
for i in range(len(submit_friends)):
  text = str(submit_friends['utterance'][i])
  #token 만드는 부분
  token = tokenizer.encode(text, max_length=max_sequence_length,  pad_to_max_length=True)
  for index, value in enumerate(token):
    if value == 1:
        token[index] = 0

  #mask 만드는 부분
  zeros = token.count(0)
  mask = [1]*(max_sequence_length-zeros) + [0]*zeros

  #segment 만드는 부분
  segment = [0]*max_sequence_length


  #각 리스트에 저장
  submit_tokens.append(token)
  submit_masks.append(mask)
  submit_segments.append(segment)

# numpy array로 변경
submit_tokens = np.array(submit_tokens)
submit_masks = np.array(submit_masks)
submit_segments = np.array(submit_segments)

submit_x = [submit_tokens, submit_masks, submit_segments]

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
preds = final_model.predict(submit_x)
print(len(preds))

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


1623


In [ ]:
submit_friends['Predicted']=''
for i in range(len(preds)):
  label = np.argmax(preds[i])
  if label == 0:
    label = 'non-neutral'
  elif label == 1:
    label = 'neutral'
  elif label == 2:
    label = 'joy'
  elif label == 3:
    label = 'sadness'
  elif label == 4:
    label = 'fear'
  elif label == 5:
    label ='anger'
  elif label == 6:
    label ='surprise'
  elif label == 7:
    label = 'disgust'
  submit_friends['Predicted'][i] = label
submit_friends.drop(['utterance'], axis='columns', inplace=True)
submit_friends.rename(columns = {'id' : 'Id'}, inplace = True)
submit_friends

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Id,Predicted
0,0,neutral
1,1,surprise
2,2,non-neutral
3,3,neutral
4,4,non-neutral
...,...,...
1618,1618,neutral
1619,1619,joy
1620,1620,neutral
1621,1621,neutral


In [ ]:
submit_friends.to_csv('/content/gdrive/My Drive/Colab Notebooks/naver_sentiment/제출데이터/friends 데이터 결과2.csv', index=False)